In [1]:
import glob
from MyCapytain.resolvers.cts.local import CtsCapitainsLocalResolver

repositories = list(glob.glob("../../data/raw/corpora/**/*", recursive=False))
resolver = CtsCapitainsLocalResolver(repositories)

../../data/raw/corpora/lascivaroma_additional-texts/additional-texts-1.0.111/data/phi1351/phi005/phi1351.phi005.perseus-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/priapeia-1.1.18/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/priapeia-1.1.18/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0472/phi001/phi0472.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0472/phi001/phi0472.phi001.perseus-eng4.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0448/phi001/phi0448.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0448/phi002/phi0448.phi002.perseus-eng2.xml is not present
../../data

../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0690/phi003/phi0690.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi1017/phi011/phi1017.phi011.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0660/phi003/phi0660.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi1002/phi001/phi1002.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0914/phi001/phi0914.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0917/phi001/phi0917.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0119/phi004/phi0119.phi004.perseus-en

../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi1351/phi001/phi1351.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi1351/phi001/phi1351.phi001.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi1351/phi002/phi1351.phi002.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/canonical-latinLit-0.0.557/data/phi0550/phi001/phi0550.phi001.perseus-eng1.xml is not present


In [3]:
import regex as re
import os.path
import glob
import tqdm

!rm ./lemmatized/*.xml
!rm ./lemmatized/xml/*.xml


TEI_START = """<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader xml:id="{tid}">
        <fileDesc>
            <titleStmt>
                <title>{title}</title>
                <author>{author}</author>
            </titleStmt>
        </fileDesc>
    </teiHeader>
    <text>
        <body>"""
TEI_END = """        </body>
    </text>
</TEI>
"""
reference = re.compile(r"(\[REF:[A-Za-z0-9\.]+\])")
for file in glob.glob("./lemmatized/tsv/*-pie.txt"):
    with open(file) as read:
        xml = file.replace("-pie.txt", ".xml")
        if xml == file:
            print(xml, file)
            break
        text_id = os.path.basename(file)[:-len("-pie.txt")]
        metadata = resolver.getMetadata(text_id)
        title = metadata.parent.get_label()
        author = metadata.parent.parent.get_label()
        with open(xml, "w") as write:
            write.write(TEI_START.format(
                tid=text_id,
                title=title,
                author=author
            ))
            for lineno, line in tqdm.tqdm(enumerate(read)):
                line = line.strip().split("\t")
                if lineno == 0:
                    headers = line
                    continue
                line = dict(zip(headers, line))
                
                
                # Deal with Segments
                if line["token"].startswith("[REF:"):
                    section, *sid = line["token"][len("[REF:"):-1].split(".")
                    sid = ".".join(sid)
                    if lineno != 1:
                        write.write("    " * 3 + "</seg>\n")
                    write.write("    " * 3 + '<seg type="{}" n="{}:{}">'.format(section, text_id, sid))
                    w_id = 1
                    continue
                    
                elif line["token"] in {"<", ">", "&"}:
                    continue
                # Deal with lemma
                write.write("    " * 4 + \
                        '<w textpart="{section}" n="{sid}" pos="{p}" lemma="{l}" msd="{m}">{t}</w>\n'.format(
                    tid=text_id, 
                    sid=sid,
                    section=section,
                    wid=w_id,
                    t=line["token"],
                    l=line["lemma"].replace('"', '&quot;'),
                    p=line["pos"],
                    m=line["morph"]
                ).replace('"&quot;"', "'\"'"))
                w_id += 1
            write.write("    " * 3 + "</seg>")
            write.write(TEI_END)
            

rm: cannot remove './lemmatized/xml/*.xml': No such file or directory


512it [00:00, 62334.44it/s]
5487it [00:00, 239364.16it/s]
469it [00:00, 283734.11it/s]
61991it [00:00, 355099.39it/s]
16189it [00:00, 345265.24it/s]
16488it [00:00, 341308.98it/s]
1646it [00:00, 322457.93it/s]
7974it [00:00, 342502.61it/s]
1080it [00:00, 300168.86it/s]
12249it [00:00, 343441.02it/s]
13290it [00:00, 331473.85it/s]
89349it [00:00, 340200.34it/s]
1478it [00:00, 286826.51it/s]
10256it [00:00, 342356.74it/s]
149125it [00:00, 346696.55it/s]
31690it [00:00, 344282.51it/s]
14200it [00:00, 338817.97it/s]
4281it [00:00, 323091.60it/s]
17778it [00:00, 342389.80it/s]
140800it [00:00, 339837.08it/s]
93143it [00:00, 324499.43it/s]
23230it [00:00, 341880.97it/s]
10700it [00:00, 342006.00it/s]
2663it [00:00, 304054.21it/s]
8114it [00:00, 327885.84it/s]
12841it [00:00, 331554.61it/s]
62207it [00:00, 343526.72it/s]
10146it [00:00, 339360.03it/s]
679it [00:00, 219629.24it/s]
13019it [00:00, 337490.61it/s]
8118it [00:00, 335147.99it/s]
13155it [00:00, 317051.00it/s]
3778it [00:00, 316832.

126650it [00:00, 328348.80it/s]
4992it [00:00, 340382.77it/s]
26218it [00:00, 341846.67it/s]
118584it [00:00, 339251.99it/s]
736it [00:00, 302469.89it/s]
55089it [00:00, 349925.51it/s]
4912it [00:00, 340727.37it/s]
15692it [00:00, 334476.86it/s]
368it [00:00, 71564.53it/s]
43091it [00:00, 353914.60it/s]
18363it [00:00, 342387.22it/s]
4146it [00:00, 334730.50it/s]
17775it [00:00, 336911.30it/s]
25203it [00:00, 351653.14it/s]
87708it [00:00, 325911.31it/s]
7153it [00:00, 340736.59it/s]
6405it [00:00, 344438.97it/s]
5033it [00:00, 336960.99it/s]
6487it [00:00, 341977.95it/s]
4493it [00:00, 338209.04it/s]
18113it [00:00, 348205.28it/s]
4379it [00:00, 343466.24it/s]
3154it [00:00, 329198.33it/s]
4607it [00:00, 336218.65it/s]
16550it [00:00, 338276.70it/s]
10464it [00:00, 334661.61it/s]
28238it [00:00, 337131.72it/s]
8276it [00:00, 344714.49it/s]
16958it [00:00, 346927.16it/s]
5128it [00:00, 337317.74it/s]
1974it [00:00, 334784.53it/s]
32527it [00:00, 316308.13it/s]
18672it [00:00, 335659.37

64100it [00:00, 326867.69it/s]
802it [00:00, 261248.20it/s]
195it [00:00, 226061.16it/s]
1878it [00:00, 327141.08it/s]
1864it [00:00, 329797.63it/s]
76529it [00:00, 346710.04it/s]
3348it [00:00, 333631.02it/s]
6535it [00:00, 358752.62it/s]
9318it [00:00, 336538.26it/s]
143004it [00:00, 337981.69it/s]
32865it [00:00, 340323.03it/s]
21814it [00:00, 343472.50it/s]
8520it [00:00, 317222.84it/s]
15470it [00:00, 341522.62it/s]
5504it [00:00, 328698.04it/s]
12782it [00:00, 334097.31it/s]
158032it [00:00, 345453.42it/s]
3354it [00:00, 329423.37it/s]
5053it [00:00, 329219.24it/s]
145260it [00:00, 331243.04it/s]
77919it [00:00, 338695.37it/s]
8045it [00:00, 333111.30it/s]
12010it [00:00, 343870.51it/s]
9405it [00:00, 316999.59it/s]
16367it [00:00, 331077.10it/s]
1509it [00:00, 316491.89it/s]
13356it [00:00, 335018.56it/s]
2677it [00:00, 314725.64it/s]
5892it [00:00, 345638.95it/s]
1853it [00:00, 314810.65it/s]
5187it [00:00, 341514.74it/s]
2882it [00:00, 317837.19it/s]
44640it [00:00, 339619.14i

mv: target 'lemmatized/xml/' is not a directory


In [4]:
!mkdir -p lemmatized/xml
!mv lemmatized/tsv/*.xml lemmatized/xml/

In [6]:
!zip data lemmatized/xml/*.xml

  adding: lemmatized/xml/urn:cts:greekLit:tlg0526.tlg003.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:anthoLat.anthoLat.musisquedeoque-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:dlt000006.001.csl-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0022.phi001.thayer-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi001.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi002.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi003.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi004.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi005.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi006.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0119.phi007.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/

  adding: lemmatized/xml/urn:cts:latinLit:phi0620.phi001.perseus-lat3.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi001.perseus-lat3.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi001.perseus-lat4.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi002.perseus-lat3.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi002.perseus-lat4.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi003.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi007.thayer-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0631.phi008.thayer-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0660.phi001.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0660.phi003.perseus-lat2.xml (deflated 89%)
  adding: lemmatized/xml/urn:cts:latinLit:phi0684.phi001.remacle-lat1.xml (deflated 93%)
  adding: lemmatized/xm

  adding: lemmatized/xml/urn:cts:latinLit:phi1351.phi004.perseus-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi1351.phi005.lascivaroma-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:phi1672.phi002.digilibLT-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2331.phi001.perseus-lat2.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2331.phi002.perseus-lat2.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2331.phi003.perseus-lat2.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2331.phi004.perseus-lat2.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2331.phi005.perseus-lat2.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2331.phi006.perseus-lat2.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2331.phi007.perseus-lat2.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:phi2331.phi008.perseus-lat2.xml (deflated 92%)
  adding: lemma

  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa041.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa042.opp-lat1.xml (deflated 94%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa042a.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa044.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa046.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa050.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa053.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa054.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa054.opp-lat2.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa055.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0040.stoa056.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:

  adding: lemmatized/xml/urn:cts:latinLit:stoa0089a.stoa002.opp-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0096.stoa003.opp-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0096.stoa004.opp-lat2.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa001.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa002.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa003.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa004.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa005.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa006.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa007.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0104a.stoa008.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:l

  adding: lemmatized/xml/urn:cts:latinLit:stoa0171.stoa002.opp-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0171.stoa004.opp-lat1.xml (deflated 89%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0171.stoa006.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0171.stoa007.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0171.stoa008.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0171.stoa009.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0171.stoa010.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0180a.stoa001.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0180a.stoa002.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0180a.stoa003.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0180a.stoa004.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinL

  adding: lemmatized/xml/urn:cts:latinLit:stoa0270.stoa005.opp-lat1.xml (deflated 91%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0270p.stoa001.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa001.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa002.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa002.opp-lat2.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa003.opp-lat1.xml (deflated 92%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa004.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa004.opp-lat2.xml (deflated 94%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa005.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa006.opp-lat1.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit:stoa0275.stoa006.opp-lat2.xml (deflated 93%)
  adding: lemmatized/xml/urn:cts:latinLit: